# Energy modeling for DP cold room
- This analysis will look at the Dispatch (DP) cold room to build an energy model and estimate potential savings using a 4 dC setpoint instead of 2 dC.
- The results will also be compared with the basic estimate used in EDA.

In [ ]:
# import libraries
import sys
import numpy as np
import pandas as pd
import math
import sklearn.metrics as sklm
import sklearn.model_selection as ms
from sklearn import preprocessing
import joblib
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
)
import plotly.io as pio
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import xgboost as xgb
import colorlover as cl
from sqlalchemy import create_engine
import shap

# configs
pd.options.display.float_format = "{:,.2f}".format
%matplotlib inline
plt.rcParams["figure.figsize"] = 10, 6
pio.templates.default = "plotly_white"

In [ ]:
# load custom functions
sys.path.append("../")
from cnrdlib import cl_ml as clml

In [ ]:
# load daily data from eda notebook
engine = create_engine(f"sqlite:///../data/RawData.db")
df = pd.read_sql_table("DailyDataEnergy", con=engine, index_col="timestamp")
df.head()

In [ ]:
# add day of the week
df["day_of_week"] = df.index.dayofweek
df.head()

In [ ]:
# check data points for high set-point
len(df[df.DP_setpoint == "high"])

In [ ]:
# check data points for low set-point
len(df[df.DP_setpoint == "low"])

- Good split of data with 53 days at the high setpoint and 61 days at the low setpoint.

# Create baseline model
- The high setpoint will be used as the baseline.
- The low setpoint will be used as the reporting period to calculate the additional energy used.

In [ ]:
# create different data frames for modelling
df_bl = df[df.DP_setpoint == "high"]
df_rp = df[df.DP_setpoint == "low"]

In [ ]:
# basic scatter plot to visually check the baseline data
sns.lmplot(
    data=df_bl,
    x="Ext_temp",
    y="DP_energy",
    lowess=True,
    line_kws={"color": "red"},
    aspect=1.6,
)
sns.kdeplot(data=df_bl, x="Ext_temp", y="DP_energy")
plt.show()

In [ ]:
# interactive scatter plot to zoom in and identify outliers
scl = cl.scales["9"]["seq"]["Blues"]
colorscale = [[float(i) / float(len(scl) - 1), scl[i]] for i in range(len(scl))]

trace = go.Scatter(
    x=df_bl.Ext_temp,
    y=df_bl.DP_energy,
    text=df_bl.index,
    mode="markers+text",
    textposition="top center",
    hoverinfo="text",
    marker=dict(opacity=0.5, sizemin=5, sizemode="area"),
)
trace_c = go.Histogram2dContour(
    x=df_bl.Ext_temp,
    y=df_bl.DP_energy,
    ncontours=5,
    colorscale=colorscale,
    showscale=False,
    opacity=0.3,
)
data = [trace, trace_c]
layout = go.Layout(title="Scatter plot")
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# remove some outliers
df_bl.drop(["2021-05-21", "2021-04-10", "2021-04-08", "2021-04-01"], inplace=True)

In [ ]:
# basic scatter plot to visually check the baseline data again :-)
sns.lmplot(
    data=df_bl,
    x="Ext_temp",
    y="DP_energy",
    lowess=True,
    line_kws={"color": "red"},
    aspect=1.6,
)
sns.kdeplot(data=df_bl, x="Ext_temp", y="DP_energy")
plt.show()

In [ ]:
# basic scatter plot to visually check the reporting period data
sns.lmplot(
    data=df_rp,
    x="Ext_temp",
    y="DP_energy",
    lowess=True,
    line_kws={"color": "red"},
    aspect=1.6,
)
sns.kdeplot(data=df_rp, x="Ext_temp", y="DP_energy")
plt.show()

In [ ]:
# feat_cols = ["DP_temp", "Ext_temp", "CDD_calc"]
feat_cols = ["DP_temp", "Ext_temp"]
print(feat_cols)

feat_labs_cols = feat_cols + ["DP_energy"]
print(feat_labs_cols)

In [ ]:
# split features into numpy array
features = np.array(df_bl[feat_cols])
print(features.shape)

In [ ]:
p = features.shape[1]
print(p)

In [ ]:
label = np.array(df_bl["DP_energy"])

In [ ]:
# train-test split
np.random.seed(4256)

indx = range(features.shape[0])
indx = ms.train_test_split(
    indx,
    test_size=0.25,
)

x_train = features[indx[0], :]
y_train = np.ravel(label[indx[0]])
x_test = features[indx[1], :]
y_test = np.ravel(label[indx[1]])

In [ ]:
# scale features
scaler = preprocessing.StandardScaler().fit(x_train[:, :])
x_train[:, :] = scaler.transform(x_train[:, :])
x_test[:, :] = scaler.transform(x_test[:, :])

# save scaler model
joblib.dump(scaler, "../models/model_feature_scaling.pkl")

x_train[:5, :]

# Linear model
Base reference

In [ ]:
x_train_constant = sm.add_constant(x_train)
x_test_constant = sm.add_constant(x_test)

In [ ]:
model_lin = sm.OLS(y_train, x_train_constant)
model_lin_fit = model_lin.fit()
model_lin_fit.summary()

In [ ]:
y_hat = model_lin_fit.predict(x_train_constant)

In [ ]:
clml.regression_metrics(y_train, y_hat, p)

In [ ]:
clml.diagnostic_plots(x_train, y_train, y_hat)

In [ ]:
y_score = model_lin_fit.predict(x_test_constant)
clml.regression_metrics(y_test, y_score, p)

In [ ]:
clml.diagnostic_plots(x_test, y_test, y_score)

# XGBoost

In [ ]:
# set up parameter grid
xgbm_param_grid = {
    "learning_rate": [0.01, 0.1, 0.3, 0.6, 0.9],
    "n_estimators": [25, 50, 100, 150],
    "subsample": [0.3, 0.5, 0.9],
    "colsample_bytree": [0.3, 0.5, 0.7],
    "gamma": [0.3, 0.5, 0.7],
    "max_depth": [3, 5, 7, 9],
    "objective": ["reg:squarederror"],
}

In [ ]:
k_fold = KFold(n_splits=3)
model_xgb = xgb.XGBRegressor()

In [ ]:
# setup grid search parameters
grid_mse = GridSearchCV(
    estimator=model_xgb,
    param_grid=xgbm_param_grid,
    scoring="neg_mean_squared_error",
    cv=k_fold,
    verbose=1,
    n_jobs=8,
)

In [ ]:
# fit model
grid_mse.fit(x_train, y_train)

In [ ]:
# print the best parameters and lowest RMSE
print(f"Best parameters found: {grid_mse.best_params_}")
print(f"Lowest RMSE found: {np.sqrt(np.abs(grid_mse.best_score_)):0.2f}")

In [ ]:
# use best parameters
model_xgb = xgb.XGBRegressor(
    colsample_bytree=0.3,
    gamma=0.5,
    learning_rate=0.3,
    max_depth=5,
    n_estimators=25,
    subsample=0.3,
)

In [ ]:
# fit model
model_xgb_fit = model_xgb.fit(x_train, y_train)

In [ ]:
y_hat = model_xgb_fit.predict(x_train)
clml.regression_metrics(y_train, y_hat, p)

In [ ]:
clml.diagnostic_plots(x_train, y_train, y_hat)

In [ ]:
y_score = model_xgb_fit.predict(x_test)
clml.regression_metrics(y_test, y_score, p)

In [ ]:
clml.diagnostic_plots(x_test, y_test, y_score)

In [ ]:
#explain the model's predictions using SHAP values
explainer = shap.TreeExplainer(model_xgb_fit)
shap_values = explainer.shap_values(x_train)

In [ ]:
#summarize the effects of all the features
shap.summary_plot(shap_values, x_train, feature_names=feat_cols)

In [ ]:
shap.summary_plot(shap_values, x_train, plot_type='bar', feature_names=feat_cols)

- XGBoost model gives a better fit and slightly better prediction

In [ ]:
# save model model
joblib.dump(model_xgb_fit, "../models/model_predict_xgboost.pkl")

In [ ]:
SEmodel = math.sqrt(sklm.mean_squared_error(y_test, y_score))
print(f"Standard error of the model is {SEmodel:0.2f}")

## Calculate energy performance

In [ ]:
# split features into numpy array
features_rp = np.array(df_rp[feat_cols])
print(features_rp.shape)

In [ ]:
# import models for reporting period
scaler = joblib.load("../models/model_feature_scaling.pkl")
model_opt = joblib.load("../models/model_predict_xgboost.pkl")

In [ ]:
# normalise features
x_rp = scaler.transform(features_rp[:, :])
x_rp[:5, :]

In [ ]:
# predict expected consumption
y_rp = model_opt.predict(x_rp)

In [ ]:
df_rp["ExpectedkWh"] = y_rp

df_rp["Residuals"] = df_rp.DP_energy - df_rp.ExpectedkWh
df_rp["CUSUM"] = df_rp.Residuals.cumsum()

# df_rp["TargetkWh"] = df_rp.ExpectedkWh * 0.97
# df_rp["ResidualsT"] = df_rp.TargetkWh - df_rp.ExpectedkWh
# df_rp["CUSUMT"] = df_rp.ResidualsT.cumsum()

print(
    "Cumulative performance against actual: {0:,.0f} kWh".format(
        df_rp["CUSUM"].tail(1).values[0]
    )
)

In [ ]:
# create cumulative energy plot
traceE = go.Scatter(
    x=df_rp.index, y=df_rp.CUSUM, name="Cumulative energy performance [kWh]"
)

layout = go.Layout(
    legend=dict(orientation="h"),
    title="Cumulative energy performance",
    xaxis=dict(
        title="",
        titlefont=dict(
            # size=18,
            color="#7f7f7f"
        ),
    ),
    yaxis=dict(
        title="Cumulative energy [kWh]",
        titlefont=dict(
            # size=18,
            color="#7f7f7f"
        ),
    ),
)
data = [traceE]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# calculate additional daily energy per day
no_of_days = len(df_rp.index)
additional_energy = df_rp["CUSUM"].tail(1).values[0]
additional_energy_per_day = additional_energy / no_of_days
print(additional_energy_per_day)

In [ ]:
print(f"Additional energy is {additional_energy_per_day:0.2f} kWh per day")
print(f"Over a year, that equates to {additional_energy_per_day*365:0.0f} kWh per annum")
print(f"That is approximately a reduction of {additional_energy_per_day*365*1.04:0.0f} kg CO2e and saving R {additional_energy_per_day*365*1.80:0.0f} per annum")

- The results from the energy model is about 70% of the average estimates using in the EDA notebook.
- Thus, using proper models to estimate savings and adjusting for the various factors is important to ensure savings are realistic to motivate for interventions.
- Another important note is that we have a few months of data only and don't cover all the seasonal variations and thus the baseline may not be representative.